In [1]:
import random
import numpy as np
import heapq
import math
from matplotlib import pylab
import random
import math

seed = 42
random.seed(seed)
np.random.seed(seed)

#Algoritmo GBFS

In [2]:
class Node:
    def __init__(self, x, y, cost):
        self.x = x
        self.y = y
        self.cost = cost
    def __lt__(self, other):
        return self.cost < other.cost

def euclidean_distance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def greedy_best_first_search(grid, start, goal):
    rows = len(grid)
    cols = len(grid[0])
    pq = []
    heapq.heappush(pq, Node(start[0], start[1], 0))
    visited = set()
    visited.add((start[0], start[1]))
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    while pq:
        current = heapq.heappop(pq)
        if (current.x, current.y) == goal:
            print(f"Goal reached at ({current.x}, {current.y})")
            return
        for d in directions:
            new_x, new_y = current.x + d[0], current.y + d[1]
            if 0 <= new_x < rows and 0 <= new_y < cols and grid[new_x][new_y] == 0 and (new_x, new_y) not in visited:
                cost = euclidean_distance(new_x, new_y, goal[0], goal[1])
                heapq.heappush(pq, Node(new_x, new_y, cost))
                visited.add((new_x, new_y))
    print("Goal not reachable")

grid = np.array([
    #[0, 1, 1, 1],
    [0, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 0, 0, 1],
    [1, 1, 0, 0]
])

start = (0, 0)
goal = (3, 3)
greedy_best_first_search(grid, start, goal)

Goal reached at (3, 3)


# Algoritmo A*

In [3]:
class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0  # Distance from start node
        self.h = 0  # Heuristic to goal
        self.f = 0  # Total cost
    def __eq__(self, other):
        return self.position == other.position
    def __lt__(self, other):
        return self.f < other.f
    def __repr__(self):
        return f"({self.position}, f: {self.f})"

def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def astar(maze, start, end):
    open_list = []
    closed_list = set()
    start_node = Node(start)
    end_node = Node(end)
    heapq.heappush(open_list, start_node)
    while open_list:
        current_node = heapq.heappop(open_list)
        closed_list.add(current_node.position)
        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) - 1) or node_position[1] < 0:
                continue
            if maze[node_position[0]][node_position[1]] != 0:
                continue
            new_node = Node(node_position, current_node)
            if new_node.position in closed_list:
                continue
            new_node.g = current_node.g + 1
            new_node.h = heuristic(new_node.position, end_node.position)
            new_node.f = new_node.g + new_node.h
            if add_to_open(open_list, new_node):
                heapq.heappush(open_list, new_node)
    return None

def add_to_open(open_list, neighbor):
    for node in open_list:
        if neighbor == node and neighbor.g > node.g:
            return False
    return True

maze = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0]
])

start = (0, 0)
end = (4, 6)
path = astar(maze, start, end)
print(path)

[(0, 0), (1, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 6), (4, 6)]


# Algoritmo genético

In [4]:
# Number of individuals in each generation
POPULATION_SIZE = 100

# Valid genes
GENES = '''abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOP
QRSTUVWXYZ 1234567890, .-;:_!"#%&/()=?@${[]}'''

# Target string to be generated
TARGET = "IA para automatizacion"

class Individual(object):
	'''
	Class representing individual in population
	'''
	def __init__(self, chromosome):
		self.chromosome = chromosome
		self.fitness = self.cal_fitness()
	@classmethod
	def mutated_genes(self):
		'''
		create random genes for mutation
		'''
		global GENES
		gene = random.choice(GENES)
		return gene
	@classmethod
	def create_gnome(self):
		'''
		create chromosome or string of genes
		'''
		global TARGET
		gnome_len = len(TARGET)
		return [self.mutated_genes() for _ in range(gnome_len)]
	def mate(self, par2):
		'''
		Perform mating and produce new offspring
		'''
		# chromosome for offspring
		child_chromosome = []
		for gp1, gp2 in zip(self.chromosome, par2.chromosome):
			# random probability
			prob = random.random()
			# if prob is less than 0.45, insert gene
			# from parent 1
			if prob < 0.45:
				child_chromosome.append(gp1)
			# if prob is between 0.45 and 0.90, insert
			# gene from parent 2
			elif prob < 0.90:
				child_chromosome.append(gp2)
			# otherwise insert random gene(mutate),
			# for maintaining diversity
			else:
				child_chromosome.append(self.mutated_genes())
		# create new Individual(offspring) using
		# generated chromosome for offspring
		return Individual(child_chromosome)

	def cal_fitness(self):
		'''
		Calculate fitness score, it is the number of
		characters in string which differ from target
		string.
		'''
		global TARGET
		fitness = 0
		for gs, gt in zip(self.chromosome, TARGET):
			if gs != gt: fitness+= 1
		return fitness

#current generation
generation = 1
found = False
population = []
# create initial population
for _ in range(POPULATION_SIZE):
			gnome = Individual.create_gnome()
			population.append(Individual(gnome))
while not found:
	# sort the population in increasing order of fitness score
	population = sorted(population, key = lambda x:x.fitness)
	# if the individual having lowest fitness score ie.
	# 0 then we know that we have reached to the target
	# and break the loop
	if population[0].fitness <= 0:
		found = True
		break
	# Otherwise generate new offsprings for new generation
	new_generation = []
	# Perform Elitism, that mean 10% of fittest population
	# goes to the next generation
	s = int((10*POPULATION_SIZE)/100)
	new_generation.extend(population[:s])
	# From 50% of fittest population, Individuals
	# will mate to produce offspring
	s = int((90*POPULATION_SIZE)/100)
	for _ in range(s):
		parent1 = random.choice(population[:50])
		parent2 = random.choice(population[:50])
		child = parent1.mate(parent2)
		new_generation.append(child)
	population = new_generation
	print("Generation: {}\tString: {}\tFitness: {}".format(generation,
		"".join(population[0].chromosome),
		population[0].fitness))
	generation += 1
print("Generation: {}\tString: {}\tFitness: {}".format(generation,
	"".join(population[0].chromosome),
	population[0].fitness))

Generation: 1	String: z{: C4&Jau1QJtBQnlVmSR	Fitness: 20
Generation: 2	String: z3,5?MVg_zt=T:yif[9LoD	Fitness: 19
Generation: 3	String: sr noMc,(atcuBta(PYooT	Fitness: 18
Generation: 4	String: ,w:n?M@ aut3e}1Y=.:{o-	Fitness: 17
Generation: 5	String: gr1K/r7 au,cGBt_(l;oo?	Fitness: 16
Generation: 6	String: gr1K/r7 au,cGBt_(l;oo?	Fitness: 16
Generation: 7	String: gr1K/r7 au,cGBt_(l;oo?	Fitness: 16
Generation: 8	String: ,w p?r@ aut9e}1_=w:-o-	Fitness: 14
Generation: 9	String: ,w p?r@ aut9e}1_=w:-o-	Fitness: 14
Generation: 10	String: ,w p?r@ aut=T}t_6a:yoW	Fitness: 12
Generation: 11	String: ,w p?r@ aut=T}t_6a:yoW	Fitness: 12
Generation: 12	String: 8w p?r@ aut9mBt0Pwgyon	Fitness: 11
Generation: 13	String: 8w p?r@ aut9mBt0Pwgyon	Fitness: 11
Generation: 14	String: I
 p?r@ autqmBti=dgbo"	Fitness: 10
Generation: 15	String: I
 p?r@ autqmBti=dgbo"	Fitness: 10
Generation: 16	String: I
 p?r@ autqmBti=dgbo"	Fitness: 10
Generation: 17	String: I
 p?r@ autom}tiPtg{on	Fitness: 8
Generation: 18	String: I

# MiniMax

In [5]:
class Tree:
    def __init__(self, children):
        self.children = children

class Terminal(Tree):
    def __init__(self, value):
        # A terminal state is a tree with no children:
        super().__init__([])
        self.value = value

def minimax(tree, maximising_player):
    if isinstance(tree, Terminal):
        return tree.value
    if maximising_player:
        max_ = float("-inf")
        for subtree in tree.children:
            max_ = max(minimax(subtree, not maximising_player), max_)
        return max_
    else:
        min_ = float("+inf")
        for subtree in tree.children:
            min_ = min(minimax(subtree, not maximising_player), min_)
        return min_

tree = Tree([ #Nodo raíz
    Tree([  # Nodo -2
        Tree([  # Nodo 3
            Terminal(-8),  # Terminal 8
            Terminal(7)   # Terminal 7
        ]),
        Tree([  # Nodo 60
            Terminal(15),  # Terminal 15
            Terminal(14)   # Terminal 14
        ])
    ]),
    Tree([  # Nodo 10
        Tree([  # Nodo 8
            Terminal(8),   # Terminal 8
            Terminal(-5)   # Terminal -5
        ]),
        Tree([  # Nodo -1
            Terminal(0),   # Terminal 0
            Terminal(1)    # Terminal 1
        ])
    ])
])

#true: jugador maximizador; false: minimizador
print(minimax(tree, True))
print(minimax(tree, False))

7
0
